In [1]:
import os
import json
import torch
import pynvml
import numpy as np
from ltp import LTP
from tqdm import tqdm
from transformers import AutoTokenizer

from settings import *
from networks.networks import *

### 选择剩余显存最大的两个GPU，如果不够则用CPU

In [2]:
# 获取传入cuda标号的剩余显存
def get_cuda_mem_remain(idx):
    handle = pynvml.nvmlDeviceGetHandleByIndex(idx)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    total_memory = info.total
    used_memory = info.used
    remaining_memory = total_memory - used_memory
    
    return remaining_memory / 1024**2   # 单位: MB

In [3]:
batch_size = 4 # 用于情感分类的批量大小
device1 = device2 = 'cpu'   # device1: 情感分类, device2: 文本摘要和命名体识别

pynvml.nvmlInit()
gpus = torch.cuda.device_count()

if(gpus == 1):
    device2 = 'cuda'

if(gpus > 1):
    mem_list = []
    for i in range(gpus):
        mem_list.append(get_cuda_mem_remain(i))
    
    if(max(mem_list) > 15000):
        batch_size = 64
    elif(max(mem_list) > 10000):
        batch_size = 32
        
    device1 = 'cuda:{}'.format(mem_list.index(max(mem_list)))
    mem_list[int(device1.split(':')[-1])] = -1
    device2 = 'cuda:{}'.format(mem_list.index(max(mem_list)))
    

pynvml.nvmlShutdown()
batch_size, device1, device2

(4, 'cpu', 'cuda')

### 导入数据与模型

In [4]:
rawdata_path = './rawdata/date_4_11.json'
suffix = rawdata_path.split('_')
suffix = suffix[-2] + '_' + suffix[-1].split('.')[0]


# 加载数据
with open(rawdata_path, encoding='utf-8') as f:
    data = json.load(f)
suffix

'4_11'

In [5]:
# 加载模型
model_root_path = './models/'
cls_tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-330M-Sentiment')
ner_tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-DeBERTa-v2-320M-Chinese', use_fast=False)
ner_tokenizer.add_special_tokens({'additional_special_tokens': list(set(POSNAME.values()))})
smy_tokenizer=AutoTokenizer.from_pretrained('IDEA-CCNL/Randeng-BART-139M-SUMMARY')

ltp = LTP().to(device2)
cls_model = torch.load(os.path.join(model_root_path, 'attitude_classify.model')).to(device1).eval()
ner_model = torch.load(os.path.join(model_root_path, 'named_entity_recognition.model')).to(device2).eval()
smy_model = torch.load(os.path.join(model_root_path, 'text_summary.model')).to(device2).eval()

### 统计事件和心态

In [6]:
EVENTS = [] # {'entities': dict, 'post': str, 'summary': str, 'time': str, 'time_hot': {str: {int: int}}}
REPEATS = []
COMMENTS = []

In [7]:
# 判断输入事件是否与事件列表中的某个事件重复(重复返回与事件列表中重复的下标, 不重复返回-1)
def is_repeat(event, thresh):
    # 获取输入事件的key_words
    key_words = []
    for k in event['entities']:
        key_words += event['entities'][k]
    key_words = set(key_words)
    
    for i, e in enumerate(EVENTS):
        # 遍历每个事件的key_words
        s = []
        for k in e['entities']:
            s += e['entities'][k]
        s = set(s)
        # 重合度匹配
        ands = key_words & s
        if(ands):
            lens = min(len(key_words), len(s))
            # 重合度大于阈值则重复
            if(len(ands) / lens > thresh):
                return i
    return -1

In [8]:
# 将命名体识别模型输出结果转化为命名体类型与key_words映射关系的字典
def res2entities(res, summary_inputs):
    summary = summary_inputs['input_ids'].cpu()[0]
    entities = {ID2ENTITY[i]: [] for i in range(1,12)}
    word = ''; last = -1
    # 遍历每个字对应的类型
    for n, i in enumerate(res.tolist()):
        # 0是None, 12是[CLS]或[PAD], 13是[SEP]
        if(i == 0 or i == 12 or i == 13):
            if(len(word) != 0 and last != -1):
                entities[ID2ENTITY[last]].append(word)
                word = ''
            last = -1
            continue
        # 出现下一个字的类型与上一个字类型不同
        if(i != last and last != -1 and len(word) != 0):
            entities[ID2ENTITY[last]].append(word)
            word = ''
        word += ner_tokenizer.decode(summary[n]) if(ner_tokenizer.decode(summary[n]).replace(' ', '') not in POSVALUES) else ''
        last = i
    # 处理最后一个字(词)
    if(i != 0 and i!= 12 and i != 13):
        entities[ID2ENTITY[i]].append(word)
    return entities

In [9]:
# 分析事件的一组评论心态
def att_cls(event, comments):
    # 定义小批量处理函数
    def collate_fn(event, data):
        inputs = cls_tokenizer.batch_encode_plus(
                [[event, c['content']] for c in data], return_tensors='pt', truncation=True, max_length=512, padding=True, add_special_tokens=True, return_token_type_ids=True
            ).to(device1)

        return data, inputs
    # 以batch的形式加载评论(避免显存溢出)
    loader = torch.utils.data.DataLoader(
        dataset=comments, batch_size=batch_size, collate_fn=lambda batch: collate_fn(event, batch), shuffle=False, drop_last=False
        )
    
    for batch_comments, inputs in loader:
        # 对batch做心态分析
        with torch.no_grad():
            out = cls_model(**inputs)['cls']
            res = out.argmax(dim=1)
            # 将该batch的心态分配到对应评论字典中
            for c in range(len(batch_comments)):
                batch_comments[c]['attitude'] = res[c].item()
        
    return comments

In [10]:
def static_item(item):
    # 获取数据
    event = item['event'].strip()
    post = item['post'].strip()
    time = item['time']
    ip = item['ip']
    thumbs = item['thumbs']
    comments = item['comments']
    
    if(post == '' or post == 'None'):
        return
    
    # 基础热度(点赞数)
    hot = thumbs
    # 文本摘要
    post_inputs = smy_tokenizer.encode_plus(post, return_tensors='pt', truncation=True, max_length=512).to(device2)
    summary = smy_tokenizer.decode(smy_model.generate(post_inputs['input_ids'], max_length=128, do_sample=False)[0]).replace('</s>', '').strip()
    # 命名体识别
    words, pos = ltp.pipeline(summary, tasks = ['cws', 'pos'], return_dict = False) 
    # 分词
    text = ''
    for i in range(len(words)):
        text = text + POSNAME[pos[i]] + words[i]
    #输入模型
    summary_inputs = ner_tokenizer.encode_plus(text=text, return_tensors="pt", padding=True, add_special_tokens=True, return_token_type_ids=False).to(device2)
    with torch.no_grad():
        out = ner_model(**summary_inputs)['cls']
        res = out.argmax(dim=1)
    # 获取关键词
    entities = res2entities(res, summary_inputs)
    event = {'entities': entities, 'post':post, 'summary': summary, 'time_hot': {'{}'.format(time): {ip: hot}}}
    idx = is_repeat(event, 0.5)
    # 对评论情感分析
    comments = att_cls(summary, comments)
    # 不重复则加入新事件和评论
    if(idx == -1):
        EVENTS.append(event)
        COMMENTS.append(comments)
    # 重复则合并
    else:
        # 合并命名体
        for i in EVENTS[idx]['entities']:
            EVENTS[idx]['entities'][i] += event['entities'][i]
            EVENTS[idx]['entities'][i] = list(set(EVENTS[idx]['entities'][i]))  # 去重
        # 如果该时间存在, 合并ip的热度
        if(EVENTS[idx]['time_hot'].get(time, None) is not None):
            EVENTS[idx]['time_hot'][time][ip] = EVENTS[idx]['time_hot'][time][ip] + hot if(EVENTS[idx]['time_hot'][time].get(ip, None) is not None) else hot
        # 不存在, 加入新的时间
        else:
            EVENTS[idx]['time_hot'].update(event['time_hot'])
        
        COMMENTS[idx] += comments
        REPEATS.append(event)

In [11]:
# 遍历data开始统计事件
for key in data.keys():
    for item in tqdm(data[key]):
        static_item(item)

100%|██████████| 316/316 [03:00<00:00,  1.75it/s]


### 保存事件和评论

In [12]:
import os

root = './output_static/events_date_{}'.format(suffix)

# 创建文件夹
if(not os.path.exists(root)):
    os.makedirs(root)
    

# 保存统计的事件
with open('{}/events_date_{}.json'.format(root, suffix), 'w', encoding='utf-8') as f:
    json.dump(EVENTS, f, ensure_ascii=False)

# 保存统计的心态
with open('{}/comments_date_{}.json'.format(root, suffix), 'w', encoding='utf-8') as f:
    json.dump(COMMENTS, f, ensure_ascii=False)

In [13]:
# 保存原句与摘要
with open(f'{root}/events_post.txt', 'w', encoding='utf-8') as f:
    for e in EVENTS:
        f.write(e['post'] + ': ' + e['summary'] + '\n')

# 保存重复的事件
with open(f'{root}/repeat.txt', 'w', encoding='utf-8') as f:
    for e in REPEATS:
        f.write(e['post'] + ': ' + e['summary'] + '\n')

### 加载事件和评论

In [14]:
import json

# 加载统计的事件
with open('{}/events_date_{}.json'.format(root, suffix), 'r', encoding='utf-8') as f:
    EVENTS = json.load(f)

# 加载统计的心态
with open('{}/comments_date_{}.json'.format(root, suffix), 'r', encoding='utf-8') as f:
    COMMENTS = json.load(f)

### 统计词频

In [15]:
import torch
import pandas as pd
from ltp import LTP
from tqdm import tqdm
from collections import Counter

In [16]:
# 获取原始数据每条帖子的原话和最新时间
def raw_data_post(path):    
    data = json.load(open(path, "r", encoding="utf-8"))
    sentences = []; time = ''
    for key in data:
        for d in data[key]:
            # 帖子
            time = max(time, d['time'])
            sentence = d['post'].strip()
            if(sentence != '' and sentence != 'None'):
                sentences.append(sentence)
    return time, sentences

time, posts = raw_data_post(rawdata_path)

##### 加载模型

In [17]:
KEY_WORDS = {'time': time, 'comments': [], 'total': []}  # 'comments': {int: dict}, 'total': {str: int}
# 加载LTP分词模型
ltp = LTP().to(device1)

#### 分词

In [18]:
def segmentation(data, batch_size):
    loader = torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, drop_last=False)

    words = [[], []]
    final_words = []
    # 批量获取分词结果和对应词性
    for inputs in loader:
        with torch.no_grad():
            tmp_words = ltp.pipeline(inputs, tasks = ['cws', 'pos'], return_dict = False)
            words[0] += tmp_words[0]
            words[1] += tmp_words[1]
            
    # 将词性并列
    for w in zip(words[0], words[1]):
        final_words += list(zip(w[0], w[1]))

    return final_words

In [19]:
# 给posts分词
total_seg = segmentation(posts, 1024)

In [20]:
# 加载标点符号集
with open('./dataset/key_words/stopwords/punctuation.txt', 'r', encoding='utf-8') as f:
    punctuation = f.readlines()
    punctuation = [w.strip() for w in punctuation]
punctuation = set(punctuation)

In [21]:
# 给所有评论分词
comments_keywords = []
for i in tqdm(range(len(COMMENTS))):
    if(COMMENTS[i] == []):
        comments_keywords.append([])
        continue
    # 每个心态的评论单独统计
    key_words = {}
    groupby = pd.DataFrame(COMMENTS[i]).groupby('attitude')
    # 遍历每个心态里面的评论
    for attitude, comments in groupby:
        comments_seg = segmentation(comments['content'].to_list(), 128)
        # 过滤
        comments_seg = list(filter(lambda x:x[0] not in punctuation, comments_seg))
        # 加入到总的关键词统计中
        total_seg += comments_seg
        # 加入到每个心态对应的统计中
        counted_seg = dict(Counter(comments_seg))
        key_words[attitude] = list([[k[0], counted_seg[k]] for k in counted_seg])

    comments_keywords.append(key_words)
    
KEY_WORDS['comments'] = comments_keywords

100%|██████████| 1362/1362 [02:52<00:00,  7.89it/s]


#### 在总关键词中过滤停用词

In [22]:
# 加载停用词
with open('./dataset/key_words/stopwords/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
    stopwords = [w.strip() for w in stopwords]
stopwords = set(stopwords)

In [23]:
# 统计总关键词
total_seg = Counter(total_seg)
KEY_WORDS['total'] = list([[k[0], k[1], total_seg[k]] for k in dict(total_seg)])
# 过滤
filted = filter(lambda x: (x[0] not in stopwords) and (x[1] in ['ns', 'n', 'nh', 'v']) and (len(x[0]) > 1), KEY_WORDS['total'])
filted = [[i[0], i[2]] for i in filted]
filted.sort(key=lambda x: int(x[1]), reverse=True)

KEY_WORDS['total'] = filted

#### 保存

In [24]:
# 保存统计的关键词
with open('{}/keywords_date_{}.json'.format(root, suffix), 'w', encoding='utf-8') as f:
    json.dump(KEY_WORDS, f, ensure_ascii=False)

### 上传数据库

#### 加载

In [23]:
ID = {'ED_ID': -1, 'EVENT_ID': -1, 'COMMENT_ID': -1, 'C_KEYWORD_ID': -1, 'T_KEYWORD_ID': -1}

In [32]:
import json

suffix = '4_5'
root = './output_static/events_date_{}'.format(suffix)

# 加载统计的事件
with open('{}/events_date_{}.json'.format(root, suffix), 'r', encoding='utf-8') as f:
    EVENTS = json.load(f)

# 加载统计的心态
with open('{}/comments_date_{}.json'.format(root, suffix), 'r', encoding='utf-8') as f:
    COMMENTS = json.load(f)
    
# 加载统计的关键词
with open('{}/keywords_date_{}.json'.format(root, suffix), 'r', encoding='utf-8') as f:
    KEY_WORDS = json.load(f)

In [33]:
import pymysql

# 从table表中获取长度为lens的未分配id_name
def get_ids(table, id_name, lens, ID, iid):
    # 如果有id的起始值, 则往后加
    begin_id = ID[iid]
    if(begin_id != -1):
        ID[iid] += lens
        return [i for i in range(begin_id, begin_id+lens)]
    
    db = pymysql.connect(host='139.155.236.234', user='root', port=3306, password='lishuai110', database='isps')
    cursor = db.cursor()
    # 获取已有的id
    cursor.execute(f'SELECT {id_name} FROM {table}')
    exist_ids = cursor.fetchall()
    exist_ids = set([i[0] for i in exist_ids])
    db.close()
    # 分配未有的id
    total_ids = set(range(1, lens + max(exist_ids)+1)) if(exist_ids) else set(range(1, lens+1))
    use_ids = list(total_ids - exist_ids)
    # 如果数据库中ID是连续的, 则直接往后加
    if(len(use_ids) == lens):
        ID[iid] = use_ids[-1] + 1
    return use_ids

In [34]:
# 插入评论统计表, 并统计点赞和占比最大的心态, 返回SQL语句, 心态, 热度
def insert_comments_statistics(comments, event_id):
    # 统计心态和点赞
    attitudes = [0 for i in range(len(ATTITUDE2ID))]
    time_hot = {}
    
    # 获取comments_id
    lens = len(comments)
    ids = get_ids('app_comments_statistics', 'comments_id', lens, ID, 'COMMENT_ID')
    
    # 构造插入数据的SQL语句, 并统计心态
    values = """VALUES"""
    for i in range(lens):
        c = comments[i]
        # 统计心态
        attitudes[c['attitude']] += 1
        # 统计点赞
        if(time_hot.get(c['time'], None) is None):
            time_hot[c['time']] = {i: 0 for i in range(len(PROVINCE2ID))}
        c['thumbs'] = c['thumbs'] if(c['thumbs'] and c['thumbs'] != 'None') else 0
        time_hot[c['time']][c['ip']] += c['thumbs']
        # 构造SQL
        values += f"""({ids[i]}, "{c['time']}", "{c['content']}", {c['ip']}, {c['attitude']}, {c['thumbs']}, {event_id}), """
    
    if(values == 'VALUES'):
        return '', attitudes.index(max(attitudes)), time_hot
    
    values = values[:-2] + ';'
    # 最终SQL
    sql = f""" INSERT INTO app_comments_statistics {values}"""
    
    return sql, attitudes.index(max(attitudes)), time_hot

In [35]:
# 将列表转化成字符串(以','隔开)
def list2string(lst):
    string = ''
    for i in lst:
        string += str(i) + ', '
        
    if(string):
        string = string[:-2]
    return string[:32]

# 将列表转化成写入SQL用的字符串
def entities2str(entities):
    names = ['事物', '机构', '动作', '数量', '人物', '地点', '原因', '物品', '时间', '触发词', '单位']
    sql = """"""
    for key in names:
        sql += '"' + list2string(entities[key]) + '", '
    return sql[:-2]

# 插入事件统计表, 返回SQL语句
def insert_event_statistics(event, event_id, attitude):
    return f""" INSERT INTO app_event_statistics 
                    VALUES({event_id}, "{event['post'][:512]}", "{event['summary']}", {attitude}, {entities2str(event['entities'])});
                    """

In [36]:
from collections import Counter

# 插入事件分布表, 返回SQL语句
def insert_event_distribution(time_hot, hot, event_id):
    # 合并time_hot和hot
    for time in hot:
        if(time_hot.get(time, None) is None):
            time_hot[time] = dict(Counter(hot[time]))
        else:
            hot_time, time_hot_time = Counter(hot[time]), Counter(time_hot[time])
            time_hot[time] = dict(hot_time + time_hot_time)
        
    # 获取id
    lens = sum([len(time_hot[i]) for i in time_hot])
    ids = get_ids('app_event_distribution', 'id', lens, ID, 'ED_ID')
    
    # 构造插入数据的SQL语句
    values = """VALUES"""; i = 0
    for time in time_hot:
        for ip in time_hot[time]:
            # 构造SQL
            values += f"""({ids[i]}, "{time}", {ip}, {time_hot[time][ip]}, {event_id}), """
            i += 1
            
    if(values == 'VALUES'):
        return ''
    
    values = values[:-2] + ';'
    # 最终SQL
    return f""" INSERT INTO app_event_distribution {values}"""

In [37]:
# 插入评论关键词表, 返回SQL语句
def insert_comment_key_words(c_keywords, time, event_id):
    lens = sum([len(c_keywords[i]) for i in c_keywords])
    # 获取id
    ids = get_ids('app_comments_key_words', 'id', lens, ID, 'C_KEYWORD_ID')
    values = """VALUES"""; i = 0
    for attitude in c_keywords:
        for key_word in c_keywords[attitude]:
            # 构造SQL
            values += f"""({ids[i]}, "{time}", "{key_word[0][:16]}", {key_word[1]}, {attitude}, {event_id}), """
            i += 1

    if(values == 'VALUES'):
        return ''
        
    values = values[:-2] + ';'
    # 最终SQL
    return f""" INSERT INTO app_comments_key_words {values}"""

In [38]:
# 插入评论统计表, 事件统计表, 事件分布表, 评论关键词表
lens = len(EVENTS)
ids = get_ids('app_event_statistics', 'event_id', lens, ID, 'EVENT_ID')

# 构造SQL
for e in tqdm(range(lens)):
    c_s_sql, attitude, hot = insert_comments_statistics(COMMENTS[e], ids[e])
    e_s_sql = insert_event_statistics(EVENTS[e], ids[e], attitude)
    e_d_sql = insert_event_distribution(EVENTS[e]['time_hot'], hot, ids[e])
    c_kw_sql = insert_comment_key_words(KEY_WORDS['comments'][e], KEY_WORDS['time'], ids[e])
    if(c_s_sql == '' or e_d_sql == '' or e_s_sql == '' or c_kw_sql == ''):
        continue
    
    # 往数据库里插入
    db = pymysql.connect(host='139.155.236.234', user='root', port=3306, password='lishuai110', database='isps')
    cursor = db.cursor()
    try:
        cursor.execute(e_s_sql)
        cursor.execute(e_d_sql)
        cursor.execute(c_s_sql)
        cursor.execute(c_kw_sql)
    except pymysql.OperationalError:
        continue
    db.commit()
    db.close()

  7%|▋         | 62/868 [00:07<01:34,  8.55it/s]


OperationalError: (1292, "Incorrect datetime value: '2022--17' for column 'event_time' at row 283")

In [ ]:
# 插入总关键词表
def insert_total_key_words(key_words, time):
    # 获取id
    lens = len(key_words)
    ids = get_ids('app_event_key_words', 'id', lens, ID, 'T_KEYWORD_ID')
    values = """VALUES"""
    for i in range(lens):
        # 构造SQL
        values += f"""({ids[i]}, "{time}", "{key_words[i][0][:16]}", {key_words[i][1]}), """
    
    if(values == 'VALUES'):
        return ''
        
    values = values[:-2] + ';'
    # 最终SQL
    return f""" INSERT INTO app_event_key_words {values}"""

# 往数据库里插入
db = pymysql.connect(host='139.155.236.234', user='root', port=3306, password='lishuai110', database='isps')
cursor = db.cursor()
t_kw_sql = insert_total_key_words(KEY_WORDS['total'], KEY_WORDS['time'])
if(t_kw_sql):
    cursor.execute(t_kw_sql)
db.commit()
db.close()